In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
## Columns to keep from data base
demigraphic = ['sitecode','year','age','sex','race4','race7','stheight','stweight','bmi']
life_style = ['qnfr0','qnfr1','qnfr2','qnfr3','qnveg0','qnveg1','qnveg2','qnveg3',
              'qnsoda1','qnsoda2','qnsoda3','qnmilk1','qnmilk2','qnmilk3','qnbk7day',
              'qnpa0day','qnpa7day']
## TODO: Maybe include data point weighting??
## TODO: Maybe extend list of kept columns
cols = demigraphic+life_style

## List of dataframes of data points sampled from each state
sample_data = []

In [3]:
## Reads in data from states A-M, dropping unnecessary columns and null rows
yrbs_a_m = pd.read_csv('databases/states_a-m1.csv')[cols].dropna(subset=['bmi']+life_style)
## NOTE: Dropping null rows removed entire states from data set
## TODO: Look into reasoning

yrbs_a_m.head()

,sitecode,year,age,sex,race4,race7,stheight,stweight,bmi,qnfr0,...,qnveg3,qnsoda1,qnsoda2,qnsoda3,qnmilk1,qnmilk2,qnmilk3,qnbk7day,qnpa0day,qnpa7day
230982,ID,2009.0,4.0,NaN,3.0,4.0,NaN,NaN,19.775087,2.0,...,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0
230983,ID,2009.0,5.0,NaN,NaN,NaN,NaN,NaN,38.163369,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
230984,ID,2009.0,NaN,2.0,1.0,6.0,NaN,NaN,28.445161,2.0,...,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0
230985,ID,2009.0,NaN,2.0,1.0,6.0,NaN,NaN,25.130194,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0
230986,ID,2009.0,1.0,2.0,4.0,5.0,1.6,66.68,26.046875,2.0,...,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0


In [4]:
## Samples 1000 data points from each state A-M
states_a_m = yrbs_a_m['sitecode'].unique()

for state in states_a_m:
    ## Not all states have 1000 data points
    ## In the case of <= 1000, all points are sampled
    if yrbs_a_m[:][yrbs_a_m.sitecode == state].shape[0] > 1000:
        sample_data.append(yrbs_a_m[:][yrbs_a_m.sitecode == state].sample(1000))
    else:
        sample_data.append(yrbs_a_m[:][yrbs_a_m.sitecode == state])

del yrbs_a_m, states_a_m

In [5]:
## Reads in data from states N-Z, dropping unnecessary columns and null rows
yrbs_n_z = pd.read_csv('databases/states_n-z1.csv')[cols].dropna()
yrbs_n_z.head()

,sitecode,year,age,sex,race4,race7,stheight,stweight,bmi,qnfr0,...,qnveg3,qnsoda1,qnsoda2,qnsoda3,qnmilk1,qnmilk2,qnmilk3,qnbk7day,qnpa0day,qnpa7day
194099,NC,2007.0,2.0,2.0,1.0,6.0,1.78,72.58,22.907461,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0
194100,NC,2007.0,2.0,2.0,1.0,6.0,1.65,70.31,25.825528,2.0,...,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0
194101,NC,2007.0,2.0,2.0,3.0,4.0,1.55,49.90,20.770031,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
194103,NC,2007.0,3.0,2.0,1.0,6.0,1.63,68.49,25.778163,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
194104,NC,2007.0,3.0,2.0,2.0,3.0,1.78,63.50,20.041661,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0


In [6]:
## Samples 1000 data points from each states N-Z
states_n_z = yrbs_n_z['sitecode'].unique()

for state in states_n_z:
    if yrbs_n_z[:][yrbs_n_z.sitecode == state].shape[0] > 1000:
        sample_data.append(yrbs_n_z[:][yrbs_n_z.sitecode == state].sample(1000))
    else:
        sample_data.append(yrbs_n_z[:][yrbs_n_z.sitecode == state])

del yrbs_n_z, states_n_z

In [7]:
## Combines all samples into one dataframe
sample_df = pd.concat(sample_data).reset_index().drop(columns={'index'})
sample_df.head()

,sitecode,year,age,sex,race4,race7,stheight,stweight,bmi,qnfr0,...,qnveg3,qnsoda1,qnsoda2,qnsoda3,qnmilk1,qnmilk2,qnmilk3,qnbk7day,qnpa0day,qnpa7day
0,ID,2011.0,5.0,2.0,1.0,6.0,1.75,67.13,21.920000,2.0,...,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0
1,ID,2011.0,4.0,2.0,4.0,7.0,1.68,60.78,21.534864,2.0,...,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0
2,ID,2013.0,6.0,1.0,1.0,6.0,1.65,49.90,18.328742,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
3,ID,2013.0,3.0,2.0,1.0,6.0,1.78,55.79,17.608257,2.0,...,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0
4,ID,2009.0,7.0,1.0,1.0,6.0,1.60,65.77,25.691406,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [8]:
sample_df.shape
## May be able to remove conditions in sampling size, check later

(31000, 26)